In [4]:
# Step 1: Install required libraries
!pip install music21 tensorflow

# Step 2: Upload trained model and encoder
from google.colab import files

print("Upload 'music_generator_model.h5' and 'note_encoder.pkl'")
uploaded = files.upload()

# Step 3: Load model and encoder
import pickle
from keras.models import load_model

model = load_model("music_generator_model.h5")
with open("note_encoder.pkl", "rb") as f:
    note_to_int = pickle.load(f)

# Step 4: Prepare reverse mapping
int_to_note = {number: note for note, number in note_to_int.items()}
n_vocab = len(note_to_int)

# Step 5: Create a random seed pattern
import numpy as np
import random

sequence_length = 100
pattern = [random.randint(0, n_vocab - 1) for _ in range(sequence_length)]
generated_notes = []

for _ in range(300):  # Generate 300 notes
    input_seq = np.reshape(pattern, (1, sequence_length, 1))
    input_seq = input_seq / float(n_vocab)

    prediction = model.predict(input_seq, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_notes.append(result)

    pattern.append(index)
    pattern = pattern[1:]

# Step 6: Convert generated notes to MIDI format
from music21 import instrument, note, chord, stream

output_notes = []
for element in generated_notes:
    if '.' in element or element.isdigit():
        notes_in_chord = element.split('.')
        chord_notes = [note.Note(int(n)) for n in notes_in_chord]
        for n in chord_notes:
            n.storedInstrument = instrument.Piano()
        output_notes.append(chord.Chord(chord_notes))
    else:
        n = note.Note(element)
        n.storedInstrument = instrument.Piano()
        output_notes.append(n)

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='generated_music.mid')

# Step 7: Download the generated MIDI file
files.download('generated_music.mid')
print("New music generated and saved as 'generated_music.mid'")


Upload 'music_generator_model.h5' and 'note_encoder.pkl'


Saving AnyConv.com__Pirates of the Caribbean X Game of Thrones  PianoViolin Cover  Eshan Denipitiya & David Loke.midi to AnyConv.com__Pirates of the Caribbean X Game of Thrones  PianoViolin Cover  Eshan Denipitiya & David Loke (5).midi


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

New music generated and saved as 'generated_music.mid'
